# Recap:

In my last post -- over a year ago -- I showed how to use linear programming to pick a simplified FPL team.

The code is below:

In [1]:
import pulp
import numpy as np

def select_team(expected_scores, prices, positions, clubs, total_budget=100, sub_factor=0.2):
    num_players = len(expected_scores)
    model = pulp.LpProblem("Constrained value maximisation", pulp.LpMaximize)
    decisions = [
        pulp.LpVariable("x{}".format(i), lowBound=0, upBound=1, cat='Integer')
        for i in range(num_players)
    ]
    captain_decisions = [
        pulp.LpVariable("y{}".format(i), lowBound=0, upBound=1, cat='Integer')
        for i in range(num_players)
    ]
    sub_decisions = [
        pulp.LpVariable("z{}".format(i), lowBound=0, upBound=1, cat='Integer')
        for i in range(num_players)
    ]


    # objective function:
    model += sum((captain_decisions[i] + decisions[i] + sub_decisions[i]*sub_factor) * expected_scores[i]
                 for i in range(num_players)), "Objective"

    # cost constraint
    model += sum((decisions[i] + sub_decisions[i]) * prices[i] for i in range(num_players)) <= total_budget  # total cost

    # position constraints
    # 1 starting goalkeeper
    model += sum(decisions[i] for i in range(num_players) if positions[i] == 1) == 1
    # 2 total goalkeepers
    model += sum(decisions[i] + sub_decisions[i] for i in range(num_players) if positions[i] == 1) == 2

    # 3-5 starting defenders
    model += sum(decisions[i] for i in range(num_players) if positions[i] == 2) >= 3
    model += sum(decisions[i] for i in range(num_players) if positions[i] == 2) <= 5
    # 5 total defenders
    model += sum(decisions[i] + sub_decisions[i] for i in range(num_players) if positions[i] == 2) == 5

    # 3-5 starting midfielders
    model += sum(decisions[i] for i in range(num_players) if positions[i] == 3) >= 3
    model += sum(decisions[i] for i in range(num_players) if positions[i] == 3) <= 5
    # 5 total midfielders
    model += sum(decisions[i] + sub_decisions[i] for i in range(num_players) if positions[i] == 3) == 5

    # 1-3 starting attackers
    model += sum(decisions[i] for i in range(num_players) if positions[i] == 4) >= 1
    model += sum(decisions[i] for i in range(num_players) if positions[i] == 4) <= 3
    # 3 total attackers
    model += sum(decisions[i] + sub_decisions[i] for i in range(num_players) if positions[i] == 4) == 3

    # club constraint
    for club_id in np.unique(clubs):
        model += sum(decisions[i] + sub_decisions[i] for i in range(num_players) if clubs[i] == club_id) <= 3  # max 3 players

    model += sum(decisions) == 11  # total team size
    model += sum(captain_decisions) == 1  # 1 captain
    
    for i in range(num_players):  
        model += (decisions[i] - captain_decisions[i]) >= 0  # captain must also be on team
        model += (decisions[i] + sub_decisions[i]) <= 1  # subs must not be on team

    model.solve()
    print("Total expected score = {}".format(model.objective.value()))

    return decisions, captain_decisions, sub_decisions

Since the 2020/21 season has just begun, let's see what team we pick for this season:

In [2]:
import pandas as pd
df = pd.read_csv(
    "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2020-21/players_raw.csv"
)
expected_scores = df["total_points"] / 38  # penalises players who played fewer games
prices = df["now_cost"] / 10
positions = df["element_type"]
clubs = df["team_code"]
# so we can read the results
names = df["first_name"] + " " + df["second_name"]

decisions, captain_decisions, sub_decisions = select_team(expected_scores.values, prices.values, positions.values, clubs.values)
player_indices = []

print()
print("First Team:")
for i in range(len(decisions)):
    if decisions[i].value() == 1:
        print("{}{}".format(names[i], "*" if captain_decisions[i].value() == 1 else ""))
        player_indices.append(i)
print()
print("Subs:")
for i in range(len(sub_decisions)):
    if sub_decisions[i].value() == 1:
        print(names[i])
        player_indices.append(i)


/home/joe/forecasting-fantasy-football/.env/lib/python3.6/site-packages/pulp/pulp.py:1195: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Total expected score = 61.82105263157895

First Team:
Ashley Westwood
Nick Pope
Virgil van Dijk
Andrew Robertson
Trent Alexander-Arnold
Kevin De Bruyne*
Anthony Martial
John Lundstram
Danny Ings
Matt Doherty
Raúl Jiménez

Subs:
Dale Stephens
Mathew Ryan
John Egan
Mark Noble


The above model selects the optimal starters, subs and captain from a set of players with perfect score forecasts and using a simplified treatment of substitutions.

This is a decent model of the problem of picking an FPL team from scratch at the beginning of a season. In reality, we have to solve the much more complex, *dynamic* problem of picking a team and planning transfers.

Let's set up a simplified task. We already have a selected team and would like to know which transfers will maximise next week's score. We can represent our selected team in the same way as before, as a vector of binary values.

Our decisions will be which players to transfer in and which to transfer out. I'm going to revert to a starting-eleven-only model for now to make this more clear.

In [3]:
num_players = 100
current_team_indices = np.random.randint(0, num_players, size=11)  # placeholder
clubs = np.random.randint(0, 20, size=100)  # placeholder
positions = np.random.randint(1, 5, size=100)  # placeholder
expected_scores = np.random.uniform(0, 10, size=100)  # placeholder

#current_sub_indices = np.random.randint(0, num_players, size=4)  # placeholder
#current_captain_indices = current_team_indices[0]  # placeholder

# convert to binary representation
current_team_decisions = np.zeros(num_players) 
current_team_decisions[current_team_indices] = 1
# convert to binary representation
#current_sub_decisions = np.zeros(num_players) 
#current_sub_decisions[current_sub_indices] = 1
# convert to binary representation
#current_captain_decisions = np.zeros(num_players) 
#current_captain_decisions[current_captain_indices] = 1

model = pulp.LpProblem("Transfer optimisation", pulp.LpMaximize)

transfer_in_decisions = [
    pulp.LpVariable("x{}".format(i), lowBound=0, upBound=1, cat='Integer')
    for i in range(num_players)
]
transfer_out_decisions = [
    pulp.LpVariable("y{}".format(i), lowBound=0, upBound=1, cat='Integer')
    for i in range(num_players)
]

next_week_team = [
    current_team_decisions[i] + transfer_in_decisions[i] - transfer_out_decisions[i]
    for i in range(num_players)
]

This takes each player and adds or removes him from the team depending on the transfer decisions. This requires a bunch of new constraints.

* Only players in the team can be transferred out
* Only players not in the team can be transferred in
* A player cannot be transferred in and out at the same time
* Players should only be transferred with others in the same position

We can program the first two in implicitly by constraining the `next_week_team` variables to zero or one. For now, instead of explicitly programming the last constraint I'm just going to have `next_week_team` satisfy the formation constraints from our earlier model. Once we include substitutes, this will automatically enforce the last rule.

In [4]:
for i in range(num_players):
    model += next_week_team[i] <= 1
    model += next_week_team[i] >= 0
    model += (transfer_in_decisions[i] + transfer_out_decisions[i]) <= 1
    
# formation constraints
# 1 starting goalkeeper
model += sum(next_week_team[i] for i in range(num_players) if positions[i] == 1) == 1

# 3-5 starting defenders
model += sum(next_week_team[i] for i in range(num_players) if positions[i] == 2) >= 3
model += sum(next_week_team[i] for i in range(num_players) if positions[i] == 2) <= 5

# 3-5 starting midfielders
model += sum(next_week_team[i] for i in range(num_players) if positions[i] == 3) >= 3
model += sum(next_week_team[i] for i in range(num_players) if positions[i] == 3) <= 5

# 1-3 starting attackers
model += sum(next_week_team[i] for i in range(num_players) if positions[i] == 4) >= 1
model += sum(next_week_team[i] for i in range(num_players) if positions[i] == 4) <= 3

# club constraint
for club_id in np.unique(clubs):
    model += sum(next_week_team[i] for i in range(num_players) if clubs[i] == club_id) <= 3  # max 3 players

model += sum(next_week_team) == 11  # total team size


We also need to deal with prices. We will make the simplifying assumption that prices do not change over time.

In [5]:
# placeholder budget and prices
budget_now = 0
buy_prices = sell_prices = np.random.uniform(4, 12, size=100)

transfer_in_cost = sum(transfer_in_decisions[i] * buy_prices[i] for i in range(num_players))
transfer_out_cost = sum(transfer_in_decisions[i] * sell_prices[i] for i in range(num_players))

budget_next_week = budget_now + transfer_out_cost - transfer_in_cost
model += budget_next_week >= 0

Finally the objective

In [6]:
# objective function:
model += sum((next_week_team[i]) * expected_scores[i]
             for i in range(num_players)), "Objective"

In [7]:
model.solve()

1

In [8]:
for i in range(num_players):
    if transfer_in_decisions[i].value() == 1:
        print("Transferred in: {} {} {}".format(i, buy_prices[i], expected_scores[i]))
    if transfer_out_decisions[i].value() == 1:
        print("Transferred out: {} {} {}".format(i, sell_prices[i], expected_scores[i]))

Transferred out: 13 5.288295423599561 3.3894008077474136
Transferred in: 21 9.070994766604375 9.37050063216644
Transferred in: 24 7.575464205312795 9.215308010669695
Transferred out: 26 6.721648486251118 3.8509953125096166
Transferred out: 29 6.742870350049889 5.270889929532872
Transferred out: 35 4.103757608625865 1.0984705677248163
Transferred out: 37 11.962272814426175 6.680666697728531
Transferred out: 38 11.935192882229016 8.812572612903082
Transferred in: 39 4.35711720422148 9.418529086420333
Transferred in: 50 4.660757831063417 9.35311785869988
Transferred in: 55 5.711563321499944 8.54701705354162
Transferred out: 57 8.783152904137925 1.032739407088693
Transferred out: 60 11.04216159716166 5.030178731633318
Transferred in: 61 4.425480373533979 8.980715758268705
Transferred in: 70 9.398762313351092 8.311564786779657
Transferred out: 76 6.4049392439866315 2.1320162654733146
Transferred in: 79 10.197071912977469 9.27892335135661
Transferred out: 92 9.530946271360687 3.6838621093883

We forgot add a penalty for making transfers! This is simple, but requires free and non-free transfers to be treated separately to maintain linearity.

Let's wrap this all up into a class `TransferOptimiser`. I've added substitues back in here, and allowed buy and sell prices to be different.

In [9]:
import sys
sys.path.append("..")
from fpl_opt.transfers import TransferOptimiser

In [10]:
num_players = 100
current_squad_indices = np.random.randint(0, num_players, size=15)
clubs = np.random.randint(0, 20, size=100)
positions = np.random.randint(1, 5, size=100)
expected_scores = np.random.uniform(0, 10, size=100)
current_squad_decisions = np.zeros(num_players) 
current_squad_decisions[current_team_indices] = 1
# placeholder budget and prices
budget_now = 0
buy_prices = sell_prices = np.random.uniform(4, 12, size=100)

opt = TransferOptimiser(expected_scores, buy_prices, sell_prices, positions, clubs)

In [11]:
transfer_in_decisions, transfer_out_decisions, starters, sub_decisions, captain_decisions = opt.solve(current_squad_indices, budget_now, sub_factor=0.2)

Solver status: 1


In [12]:
for i in range(num_players):
    if transfer_in_decisions[i].value() == 1:
        print("Transferred in: {} {} {}".format(i, buy_prices[i], expected_scores[i]))
    if transfer_out_decisions[i].value() == 1:
        print("Transferred out: {} {} {}".format(i, sell_prices[i], expected_scores[i]))

Transferred out: 4 5.523518914053757 5.8811122126838615
Transferred in: 11 9.68275322440353 8.724190592378312
Transferred out: 13 9.38029311134498 5.528761909687386
Transferred in: 19 7.601695371029728 7.685260823699736
Transferred out: 21 5.061788918455829 3.3968564864217776
Transferred out: 22 11.806301982605639 2.8725605956680242
Transferred out: 28 7.942472393428144 6.4075948640517035
Transferred in: 43 5.315761752333296 8.9917176840911
Transferred out: 44 10.326213682210348 1.9597534343030376
Transferred in: 63 5.645606754396033 8.54310901245048
Transferred in: 64 4.745211492536894 9.039615643303172
Transferred in: 67 6.854766060238449 9.68020878348359
Transferred in: 77 8.239379570498398 9.901310186670342
Transferred in: 83 4.245940106874752 9.208222133026297
Transferred out: 84 4.799905574464629 1.8295237733961356


Let's apply this to my current team, again using a very simple player score forecast.

In [61]:
import pandas as pd
df = pd.read_csv(
    "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2020-21/players_raw.csv"
)
expected_scores = df["total_points"] / 38  # penalises players who played fewer games
prices = df["now_cost"] / 10
positions = df["element_type"]
clubs = df["team_code"]
# so we can read the results
names = df["first_name"] + " " + df["second_name"]

decisions, captain_decisions, sub_decisions = select_team(expected_scores, prices.values, positions.values, clubs.values)
player_indices = []

print()
print("First Team:")
for i in range(len(decisions)):
    if decisions[i].value() == 1:
        print("{}{}".format(names[i], "*" if captain_decisions[i].value() == 1 else ""))
        player_indices.append(i)
print()
print("Subs:")
for i in range(len(sub_decisions)):
    if sub_decisions[i].value() == 1:
        print(names[i])
        player_indices.append(i)

Total expected score = 61.82105263157895

First Team:
Ashley Westwood
Nick Pope
Virgil van Dijk
Andrew Robertson
Trent Alexander-Arnold
Kevin De Bruyne*
Anthony Martial
John Lundstram
Danny Ings
Matt Doherty
Raúl Jiménez

Subs:
Dale Stephens
Mathew Ryan
John Egan
Mark Noble


In [62]:
# next week score forecast: start with points-per-game
score_forecast = df["total_points"] / 38
# let's make up a nonsense forecast to add some dynamics -- maybe we expect some teams to do well:
score_forecast.loc[df["team"].isin(range(1,21,2))] += 1
# and some to do badly
score_forecast.loc[df["team"].isin(range(2,22,2))] -= 1
score_forecast = score_forecast.fillna(0)

In [63]:
opt = TransferOptimiser(score_forecast.values, prices.values, prices.values, positions.values, clubs.values)
transfer_in_decisions, transfer_out_decisions, starters, sub_decisions, captain_decisions = opt.solve(player_indices, budget_now=0, sub_factor=0.2)

Solver status: 1


In [66]:
for i in range(len(transfer_in_decisions)):
    if transfer_in_decisions[i].value() == 1:
        print("Transferred in: {} {} {}".format(names[i], prices[i], score_forecast[i]))
    if transfer_out_decisions[i].value() == 1:
        print("Transferred out: {} {} {}".format(names[i], prices[i], score_forecast[i]))

Transferred in: Richarlison de Andrade 8.0 5.342105263157895
Transferred out: Raúl Jiménez 8.5 4.105263157894737


In [67]:
player_indices = []
print()
print("First Team:")
for i in range(len(starters)):
    if starters[i].value() == 1:
        print("{}{}".format(names[i], "*" if captain_decisions[i].value() == 1 else ""))
        player_indices.append(i)
print()
print("Subs:")
for i in range(len(sub_decisions)):
    if sub_decisions[i].value() == 1:
        print(names[i])
        player_indices.append(i)


First Team:
Mathew Ryan
Richarlison de Andrade
Virgil van Dijk
Andrew Robertson
Trent Alexander-Arnold*
Kevin De Bruyne
Anthony Martial
John Egan
John Lundstram
Matt Doherty
Mark Noble

Subs:
Dale Stephens
Ashley Westwood
Nick Pope
Danny Ings
